# Training model for aggressive behavior prediction based on SEPP

COMBIOS RESEARCH GROUP


In [1]:
#Open_CP
!pip install https://github.com/QuantCrimAtLeeds/PredictCode/zipball/master
#SEPP
!pip install https://github.com/MatthewDaws/SEPP/zipball/master

  Using cached https://github.com/QuantCrimAtLeeds/PredictCode/zipball/master
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://github.com/MatthewDaws/SEPP/zipball/master
You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import datetime
import numpy as np
import pyproj
import dateutil.parser
import descartes
import math
import csv

import matplotlib.pyplot as plt
import matplotlib.collections

# Librerias creadas por QuantCrimAtLeeds grupo de investigacion crimen de la universidad de Leeds U.K.
# https://github.com/QuantCrimAtLeeds/PredictCode
import open_cp
import open_cp.plot
import open_cp.geometry
import open_cp.predictors
import open_cp.sources.sepp

import open_cp.seppexp as seppexp
from open_cp import evaluation
import open_cp.sepp as sepp

Failed to import `rtree`.
Failed to import `rtree`.


In [5]:
def selectData(df,nameLoc,dateIni,dateEnd):
    dfloc = df[df["LOCALIDAD"]==nameLoc]
    dfloc["FECHA_HORA"]=dfloc["HORA"].astype(str)
    valHour = dfloc["FECHA_HORA"].values
    valDate = dfloc["FECHA"].values
    timesVals = [];
    k = 0;
    for i in valHour:
        if len(i)<=2:
            timeStr = valDate[k] +" " + "00:"+i+":00"
        else:
            timeStr = valDate[k] +" " + i[:-2]+":"+i[-2:]+":00"
        
        k = k + 1

        timesVals.append(timeStr)
    dfloc["FECHA_HORA"] = timesVals;
    
    dfloc["FECHA_HORA_TS"] = dfloc.FECHA_HORA.map(dateutil.parser.parse)
    dfFilter = dfloc[(dfloc.FECHA_HORA_TS > dateIni) & (dfloc.FECHA_HORA_TS < dateEnd)]
    timestamps = dfFilter.FECHA_HORA_TS
    
    print("TimeStamps")
    print(type(timestamps))

    xcoords, ycoords = (dfFilter.LONGITUD.values,dfFilter.LATITUD.values)
    proj = pyproj.Proj(init="EPSG:3116")
    xcoords, ycoords = proj(xcoords,ycoords)
    
    points_train = open_cp.TimedPoints.from_coords(timestamps, xcoords, ycoords)
    return (points_train,timestamps,xcoords, ycoords)

def trainModel(df,nameLoc,dateIni,dateEnd):
    points_train,timestamps,xcoords, ycoords = selectData(df,nameLoc,dateIni,dateEnd)
    import matplotlib.pyplot as plt
    plt.scatter(xcoords, ycoords)
    print([timestamps])
    trainer = sepp.SEPPTrainer()
    trainer.data = points_train
    trainer.space_cutoff = 1000
    trainer.time_cutoff = datetime.timedelta(days=90)
    print(len(xcoords))
    predictor = trainer.train()
    return predictor

def trainModel_1(df,nameLoc,dateIni,dateEnd):
    points_train,timestamps,xcoords, ycoords = selectData(df,nameLoc,dateIni,dateEnd)
    import matplotlib.pyplot as plt
    plt.scatter(xcoords, ycoords)
    print([timestamps])
    trainer = sepp.SEPPTrainer()
    trainer.data = points_train    
    while True:
        try:
            predictor = trainer.train()
            break
        except (RuntimeError, ValueError) as e:
            print(e)
            pass
    return predictor 

def trainModel_2(df,nameLoc,dateIni,dateEnd):
    points_train,timestamps,xcoords, ycoords = selectData(df,nameLoc,dateIni,dateEnd)
    import matplotlib.pyplot as plt
    plt.scatter(xcoords, ycoords)
    print([timestamps])
    trainer = sepp.SEPPTrainer()
    trainer.data = points_train
    trainer.space_cutoff = 1000
    trainer.time_cutoff = datetime.timedelta(days=90)
    while True:
        try:
            predictor = trainer.train()
            break
        except (RuntimeError, ValueError) as e:
            print(e)
            pass
    return predictor 